In [1]:
# Step 1: Import the necessary packages
try:
    import sys
    import onnx
    import onnxruntime as ort
    from onnxscript import FLOAT, script, opset15 as op  # Using FLOAT for type annotation
    import numpy as np
    import os
    print("Step 1: Importing onnx, onnxruntime, onnxscript, numpy, and os passed.")
except ImportError as e:
    print(f"Step 1 failed: {str(e)}")
    sys.exit(1)
except Exception as e:
    print(f"Step 1 failed with an unexpected error: {str(e)}")
    sys.exit(1)

# Step 2: Create an ONNX model using onnxscript
try:
    @script()
    def add_model(x: FLOAT[1, 3, 224, 224], y: FLOAT[1, 3, 224, 224]) -> FLOAT[1, 3, 224, 224]:
        return op.Add(x, y)

    onnx_model = add_model.to_model_proto()
    print("Step 2: Creating an ONNX model using onnxscript passed.")
except Exception as e:
    print(f"Step 2 failed: {str(e)}")
    sys.exit(1)

# Step 3: Validate the created ONNX model
try:
    onnx.checker.check_model(onnx_model)
    print("Step 3: Model validation passed.")
except onnx.checker.ValidationError as e:
    print(f"Step 3 failed: Model validation error - {str(e)}")
    sys.exit(1)
except Exception as e:
    print(f"Step 3 failed: {str(e)}")
    sys.exit(1)

# Step 4: Save the ONNX model to a file
try:
    model_path = 'onnxscript_add_model.onnx'
    onnx.save(onnx_model, model_path)
    assert os.path.exists(model_path), "Model save failed, file not created."
    print(f"Step 4: Saving the ONNX model to '{model_path}' passed.")
except Exception as e:
    print(f"Step 4 failed: {str(e)}")
    sys.exit(1)

# Step 5: Perform inference using ONNX Runtime
try:
    # Create an ONNX Runtime session
    session = ort.InferenceSession(model_path)
    
    # Generate dummy inputs matching the model's input shape
    dummy_input1 = np.random.randn(1, 3, 224, 224).astype(np.float32)
    dummy_input2 = np.random.randn(1, 3, 224, 224).astype(np.float32)
    
    # Perform inference
    input_name1 = session.get_inputs()[0].name
    input_name2 = session.get_inputs()[1].name
    output_name = session.get_outputs()[0].name
    result = session.run([output_name], {input_name1: dummy_input1, input_name2: dummy_input2})
    
    assert result is not None, "Inference failed, no output received."
    assert np.allclose(result, dummy_input1 + dummy_input2), "Inference result is not as expected."
    print("Step 5: Performing inference using ONNX Runtime passed.")
except Exception as e:
    print(f"Step 5 failed: {str(e)}")
    sys.exit(1)

# Step 6: Explore ONNX Runtime session details
try:
    # Check input and output details
    input_details1 = session.get_inputs()[0]
    input_details2 = session.get_inputs()[1]
    output_details = session.get_outputs()[0]
    
    assert input_details1.shape == [1, 3, 224, 224], f"Unexpected input shape: {input_details1.shape}"
    assert input_details2.shape == [1, 3, 224, 224], f"Unexpected input shape: {input_details2.shape}"
    assert output_details.shape == [1, 3, 224, 224], f"Unexpected output shape: {output_details.shape}"
    
    print("Step 6: Exploring ONNX Runtime session details passed.")
    print(f"Input details: {input_details1}")
    print(f"Output details: {output_details}")
except Exception as e:
    print(f"Step 6 failed: {str(e)}")
    sys.exit(1)

# Final Confirmation
print("All extensive tests for the 'onnxscript' package completed successfully.")


Step 1: Importing onnx, onnxruntime, onnxscript, numpy, and os passed.
Step 2: Creating an ONNX model using onnxscript passed.
Step 3: Model validation passed.
Step 4: Saving the ONNX model to 'onnxscript_add_model.onnx' passed.
Step 5: Performing inference using ONNX Runtime passed.
Step 6: Exploring ONNX Runtime session details passed.
Input details: NodeArg(name='x', type='tensor(float)', shape=[1, 3, 224, 224])
Output details: NodeArg(name='return_val', type='tensor(float)', shape=[1, 3, 224, 224])
All extensive tests for the 'onnxscript' package completed successfully.
